# Sample client code for CI services

This is a sample notebook to drive your CI services from Jupyter Notebook.

## Description

This example demonstrates how to use services deployed using the new CI service template.  As an example, we use heat diffusion service.

### Questions?
Please send them to Kei Ono (kono at ucsd edu)

----

## Step 0: Boilarplate code for service callers

__Find your container host IP address first!__

In [1]:
import requests
import json

# Utility function to print JSON nicely in notebooks
def jprint(data):
    print(json.dumps(data, indent=4))

IP = '192.168.99.100'
BASE = 'http://' + IP + '/v1'  # You need to change this to your service server
HEADERS = {'Content-Type': 'application/json'}

print ('Click the link below to make sure your service can be called from web browser:\n')
print(BASE)

# Test Service from notebook
res = requests.get(BASE)
print('\nIt should be same as below...\n')
jprint(res.json())

BASE = BASE + '/' 

Click the link below to make sure your service can be called from web browser:

http://192.168.99.100/v1

It should be same as below...

{
    "version": "1",
    "message": "Called by:  192.168.99.1",
    "serviceName": "Cytoscape CI template service"
}


## Step 1: Check Available Services

In [3]:
# This is for listing all registered services
res = requests.get(BASE + 'services')
print("Here is the list of services supported by this submit agent:\n")
jprint(res.json())

Here is the list of services supported by this submit agent:

[
    {
        "parameters": {
            "network_url": {
                "required": true,
                "description": "URL of the source SIF network file.",
                "type": "string"
            }
        },
        "description": "Kernel generator service.",
        "portNumber": "5556",
        "serviceName": "kernel"
    },
    {
        "parameters": [
            {
                "required": true,
                "description": "arg",
                "type": "string",
                "name": "arg"
            }
        ],
        "description": "Shell script worker example",
        "portNumber": "5548",
        "serviceName": "shell"
    },
    {
        "parameters": {
            "network_url": {
                "required": true,
                "description": "URL of the SIF network file for the kernel.",
                "type": "string"
            },
            "query": {
                "required

## Sample1: Find Subnetwork using pre-computed kernel file

if you already have pre-computed kernel file, you can use it to find sub networks using:

* URL of Kernel file
* URL of  SIF (network) file

### _f (kernel, network, query) = subnetwork_

In [4]:
# List of sample file locations
small_network = 'https://s3-us-west-2.amazonaws.com/ci-service-data/small.sif'
med_network = 'https://s3-us-west-2.amazonaws.com/ci-service-data/yeastHighQuality.sif'

# Pre-computed sample kernel and network (HumanNet) stored in UCSD web server (about 0.5 GB)
human_net = 'http://chianti.ucsd.edu/~kono/ci/data/HumanNet.v1.symbol.sif'
human_net_kernel = 'http://chianti.ucsd.edu/~kono/ci/data/human_net.kernel.symbol.txt'

# Specify kernel and SIF file locations
base_query = {
    'kernel_url': human_net_kernel,
    'network_url': human_net,
}

gene_list = ['BRCA1', 'MAPK1']
base_query['query'] = gene_list

res = requests.post(BASE + 'services/subnet', data=json.dumps(base_query), headers=HEADERS)
jprint(res.json())
job_id1 = res.json()['job_id']

{
    "job_id": "07df4c64-6402-48af-a9d0-54f9e83cebee",
    "status": "queued"
}


In [5]:
# Check the status of the job
res = requests.get(BASE + 'queue')
jprint(res.json())

[
    {
        "job_id": "a65bfa0d-072b-4f19-a66f-2e11aa7b2326",
        "status": "finished"
    },
    {
        "job_id": "89e4415c-2ca7-417c-805a-dfe8497854d8",
        "status": "finished"
    },
    {
        "job_id": "bbf6edfc-e6ae-44f7-aa99-4f2b06d06812",
        "status": "finished"
    },
    {
        "job_id": "9e8cac38-325d-4381-a69a-5bfee0c22a06",
        "status": "finished"
    },
    {
        "job_id": "34f313a2-8161-44f0-b14d-208b77ff5d38",
        "status": "finished"
    },
    {
        "job_id": "6d7c351d-e3e1-430c-941f-1d3b2fc03262",
        "status": "finished"
    },
    {
        "job_id": "e945a7c9-a8cf-4592-ab16-c0d71b3d64c2",
        "status": "finished"
    },
    {
        "job_id": "8f2670c0-f841-42fe-9b7a-4a2205487c05",
        "status": "finished"
    },
    {
        "job_id": "2d8810bc-2734-4269-a804-8e51e417d06f",
        "status": "finished"
    },
    {
        "job_id": "07df4c64-6402-48af-a9d0-54f9e83cebee",
        "status": "running"
    },

In [6]:
# Get the result (Subnetwork in CX format)
result_url = BASE + 'queue/' + '07df4c64-6402-48af-a9d0-54f9e83cebee' + '/result'
print(result_url)
res = requests.get(result_url)
#jprint(res.json())

http://192.168.99.100/v1/queue/07df4c64-6402-48af-a9d0-54f9e83cebee/result


## Sample 2: Create kernel from SIF file

### _f (network) = kernel_

In [ ]:
# Specify locations of the SIF files
sif_file_locations = [small_network, med_network]

kernel_generation_tasks = {}

for sif_location in sif_file_locations:
    body = {
        'network_url': sif_location
    }
    res = requests.post(BASE + 'services/kernel', json=body, headers=HEADERS)
    kernel_generation_tasks[sif_location] = res.json()['job_id']
    jprint(res.json())

In [ ]:
# Check the status of the job
res = requests.get(BASE + 'queue')
jprint(res.json())

In [ ]:
# Get result
job_id_for_small_network = kernel_generation_tasks[small_network]

kernel_url = BASE + 'queue/' + job_id_for_small_network + '/result'
print(kernel_url)
res = requests.get(kernel_url)
kernel_info = res.json()
jprint(kernel_info)

In [ ]:
# Use the result to find subnetwork
base_query = {
    'kernel_url': kernel_info['kernel_file'],
    'network_url': kernel_info['network']
}

gene_list = ["NRAS", "KRAS", "MAPK1"]
base_query['query'] = gene_list

res = requests.post(BASE + 'services/subnet', data=json.dumps(base_query), headers=HEADERS)
find_subnet_job_id = res.json()['job_id']
jprint(res.json())

In [ ]:
# Check the result
result_url = BASE + 'queue/' + find_subnet_job_id + '/result'
print(result_url)
res = requests.get(result_url)
#jprint(res.json())

## Sample 3: Get actual Kernel file
For current configuration, kernels will be saved in a server in a restricted zone (= docker container separated from internet zone).  To access this, you need to open port. 

__Of course, this may be changed. It depends on deployment target.__


In [ ]:
import pandas as pd

temp = kernel_info['kernel_file']
parts = temp.split('/')
parts[2] = IP + ':3333' # This is the default opened port...
kernel_url = '/'.join(parts)
print(kernel_url)

df = pd.read_csv(kernel_url, sep="\t")
df.head()